# Face Mask Detection with MobileNetV2 🎭

Bu proje, derin öğrenme ve bilgisayar görüsü kullanarak yüz maskesi tespiti yapan bir sistem geliştirmeyi amaçlamaktadır. MobileNetV2 mimarisi kullanarak transfer learning yapılmış ve Kaggle'dan alınan veri seti ile eğitilmiştir.

## Proje Özellikleri
- **Model**: MobileNetV2 (Transfer Learning)
- **Veri Seti**: Kaggle - Face Mask Dataset
- **Sınıflar**: with_mask, without_mask
- **Görüntü Boyutu**: 224x224
- **Eğitim/Test Oranı**: 80/20

## Amaç
Bir kişinin yüz maskesi takıp takmadığını otomatik olarak tespit etmek.

## 1. Gerekli Kütüphanelerin Yüklenmesi ve İçe Aktarılması

Projeye başlamadan önce gerekli tüm kütüphaneleri yükleyelim.

In [ ]:
# Temel kütüphaneler
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import pickle
import warnings
warnings.filterwarnings('ignore')

# Machine Learning kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

# TensorFlow ve Keras kütüphaneleri
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("TensorFlow versiyonu:", tf.__version__)
print("Kullanılabilir GPU:", tf.config.list_physical_devices('GPU'))

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 2. Veri Setinin İndirilmesi ve Klasör Yapısının İncelenmesi

Veri setimiz Kaggle'dan alınan Face Mask Dataset'tir. Veri setinin yapısını inceleyelim.

In [ ]:
# Veri seti dizin yapısı
data_dir = 'data'
categories = ['with_mask', 'without_mask']

# Veri setinin mevcut olup olmadığını kontrol et
if os.path.exists(data_dir):
    print("✅ Veri seti klasörü bulundu!")
    
    # Her kategori için görüntü sayısını say
    for category in categories:
        category_path = os.path.join(data_dir, category)
        if os.path.exists(category_path):
            image_count = len(os.listdir(category_path))
            print(f"📁 {category}: {image_count} görüntü")
        else:
            print(f"❌ {category} klasörü bulunamadı!")
    
    # Toplam görüntü sayısı
    total_images = sum([len(os.listdir(os.path.join(data_dir, cat))) 
                       for cat in categories if os.path.exists(os.path.join(data_dir, cat))])
    print(f"\n📊 Toplam görüntü sayısı: {total_images}")
    
else:
    print("❌ Veri seti klasörü bulunamadı!")
    print("Lütfen Kaggle'dan veri setini indirin ve 'data' klasörüne çıkarın.")
    print("Veri seti bağlantısı: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset")

## 3. Veri Ön İşleme ve Görselleştirme

Görüntüleri yükleyip ön işlemden geçirelim ve örnek görselleri görselleştirelim.

In [ ]:
def load_data(data_dir, img_size=(224, 224)):
    """
    Veri setini yükler ve ön işlemden geçirir.
    """
    print("📥 Veri seti yükleniyor...")
    data = []
    labels = []
    
    for category in categories:
        category_path = os.path.join(data_dir, category)
        if not os.path.exists(category_path):
            continue
            
        print(f"🔄 {category} kategorisi işleniyor...")
        
        for i, img_name in enumerate(os.listdir(category_path)):
            if i % 500 == 0:
                print(f"   İşlenen görüntü: {i}")
                
            img_path = os.path.join(category_path, img_name)
            
            try:
                # Görüntüyü yükle
                image = cv2.imread(img_path)
                if image is None:
                    continue
                    
                # BGR'dan RGB'ye dönüştür
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                # Boyutlandır
                image = cv2.resize(image, img_size)
                
                # Array'e dönüştür
                image = img_to_array(image)
                
                # Normalize et
                image = preprocess_input(image)
                
                data.append(image)
                labels.append(category)
                
            except Exception as e:
                print(f"   Hata: {img_path} - {str(e)}")
                continue
    
    print(f"✅ Toplam {len(data)} görüntü yüklendi.")
    return np.array(data), np.array(labels)

# Veri setini yükle
if os.path.exists(data_dir):
    data, labels = load_data(data_dir)
    print(f"\n📊 Veri seti şekli: {data.shape}")
    print(f"📊 Etiket sayısı: {len(labels)}")
    print(f"📊 Etiket dağılımı:")
    unique, counts = np.unique(labels, return_counts=True)
    for label, count in zip(unique, counts):
        print(f"   {label}: {count} görüntü")
else:
    print("❌ Veri seti yüklenemedi!")

In [ ]:
# Örnek görüntüleri görselleştir
def visualize_samples(data, labels, num_samples=8):
    """
    Veri setinden örnek görüntüleri görselleştirir.
    """
    plt.figure(figsize=(15, 8))
    
    for i in range(num_samples):
        plt.subplot(2, 4, i+1)
        
        # Görüntüyü normalize et (görselleştirme için)
        img = data[i].copy()
        img = img - img.min()
        img = img / img.max()
        
        plt.imshow(img)
        plt.title(f"Etiket: {labels[i]}", fontsize=12)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Veri seti dağılımını görselleştir
def plot_class_distribution(labels):
    """
    Sınıf dağılımını görselleştirir.
    """
    unique, counts = np.unique(labels, return_counts=True)
    
    plt.figure(figsize=(10, 6))
    colors = ['#FF9999', '#66B2FF']
    
    # Bar grafiği
    plt.subplot(1, 2, 1)
    bars = plt.bar(unique, counts, color=colors)
    plt.title('Sınıf Dağılımı', fontsize=14, fontweight='bold')
    plt.xlabel('Sınıflar')
    plt.ylabel('Görüntü Sayısı')
    
    # Değerleri bar üzerine yazdır
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
                str(count), ha='center', fontsize=12, fontweight='bold')
    
    # Pasta grafiği
    plt.subplot(1, 2, 2)
    plt.pie(counts, labels=unique, autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title('Sınıf Oranları', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Görselleştirmeleri çalıştır
if 'data' in locals() and 'labels' in locals():
    print("🎨 Örnek görüntüler:")
    visualize_samples(data, labels)
    
    print("\n📊 Sınıf dağılımı:")
    plot_class_distribution(labels)
else:
    print("❌ Veri seti henüz yüklenmedi!")

## 4. Veri Setinin Eğitim ve Test Olarak Bölünmesi

Veri setimizi %80 eğitim ve %20 test olarak böleceğiz.

In [ ]:
# Etiketleri binary formata çevir
label_map = {'without_mask': 0, 'with_mask': 1}

if 'labels' in locals():
    # Etiketleri sayısal değerlere dönüştür
    y = np.array([label_map[label] for label in labels])
    
    print("🔄 Veri seti bölünüyor...")
    
    # Train-test split (80-20)
    X_train, X_test, y_train, y_test = train_test_split(
        data, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"✅ Veri seti başarıyla bölündü!")
    print(f"📊 Eğitim seti: {X_train.shape[0]} görüntü")
    print(f"📊 Test seti: {X_test.shape[0]} görüntü")
    
    # Eğitim ve test setlerindeki sınıf dağılımını kontrol et
    print(f"\n📊 Eğitim seti sınıf dağılımı:")
    unique_train, counts_train = np.unique(y_train, return_counts=True)
    for label_num, count in zip(unique_train, counts_train):
        label_name = 'with_mask' if label_num == 1 else 'without_mask'
        print(f"   {label_name}: {count} görüntü")
    
    print(f"\n📊 Test seti sınıf dağılımı:")
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    for label_num, count in zip(unique_test, counts_test):
        label_name = 'with_mask' if label_num == 1 else 'without_mask'
        print(f"   {label_name}: {count} görüntü")
        
    # Veri setinin şekillerini görselleştir
    plt.figure(figsize=(12, 5))
    
    # Eğitim seti dağılımı
    plt.subplot(1, 2, 1)
    train_labels = ['without_mask', 'with_mask']
    plt.bar(train_labels, counts_train, color=['#FF9999', '#66B2FF'])
    plt.title('Eğitim Seti Dağılımı', fontsize=14, fontweight='bold')
    plt.ylabel('Görüntü Sayısı')
    for i, count in enumerate(counts_train):
        plt.text(i, count + 10, str(count), ha='center', fontsize=12)
    
    # Test seti dağılımı
    plt.subplot(1, 2, 2)
    test_labels = ['without_mask', 'with_mask']
    plt.bar(test_labels, counts_test, color=['#FF9999', '#66B2FF'])
    plt.title('Test Seti Dağılımı', fontsize=14, fontweight='bold')
    plt.ylabel('Görüntü Sayısı')
    for i, count in enumerate(counts_test):
        plt.text(i, count + 2, str(count), ha='center', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ Veri seti henüz yüklenmedi!")

## 5. Veri Artırma (Data Augmentation)

Eğitim performansını artırmak için veri artırma tekniklerini uygulayalım.

In [ ]:
# Veri artırma parametrelerini tanımla
print("🔄 Veri artırma (Data Augmentation) ayarları:")

# Eğitim için veri artırma
train_datagen = ImageDataGenerator(
    rotation_range=20,          # 20 derece döndürme
    zoom_range=0.15,            # %15 zoom
    width_shift_range=0.2,      # Genişlik kayması
    height_shift_range=0.2,     # Yükseklik kayması
    shear_range=0.15,           # Kesme dönüşümü
    horizontal_flip=True,       # Yatay çevirme
    fill_mode="nearest",        # Doldurma modu
    rescale=1./255             # Normalizasyon
)

# Test için sadece normalizasyon
test_datagen = ImageDataGenerator(rescale=1./255)

print("✅ Veri artırma ayarları hazırlandı!")

# Veri artırma örneklerini görselleştir
def visualize_augmentation(X, y, datagen, num_samples=8):
    """
    Veri artırma örneklerini görselleştirir.
    """
    plt.figure(figsize=(16, 8))
    
    # Bir örnek seç
    sample_idx = np.random.randint(0, len(X))
    sample_image = X[sample_idx:sample_idx+1]
    sample_label = y[sample_idx]
    
    # Orijinal görüntü
    plt.subplot(2, 4, 1)
    img = sample_image[0].copy()
    img = (img - img.min()) / (img.max() - img.min())
    plt.imshow(img)
    plt.title(f"Orijinal\\nEtiket: {'Mask' if sample_label == 1 else 'No Mask'}")
    plt.axis('off')
    
    # Artırılmış görüntüler
    i = 0
    for batch in datagen.flow(sample_image, batch_size=1):
        i += 1
        if i >= num_samples:
            break
        
        plt.subplot(2, 4, i+1)
        img = batch[0]
        plt.imshow(img)
        plt.title(f"Artırılmış {i}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Veri artırma örneklerini göster
if 'X_train' in locals():
    print("\\n🎨 Veri artırma örnekleri:")
    visualize_augmentation(X_train, y_train, train_datagen)
else:
    print("❌ Eğitim verisi henüz hazırlanmadı!")

## 6. Modelin Oluşturulması (MobileNetV2 ile Transfer Learning)

MobileNetV2 mimarisini kullanarak transfer learning ile modelimizi oluşturalım.

In [ ]:
def create_model(input_shape=(224, 224, 3), learning_rate=0.001):
    """
    MobileNetV2 tabanlı face mask detection modeli oluşturur.
    """
    print("🔄 Model oluşturuluyor...")
    
    # MobileNetV2 base model (ImageNet ağırlıkları ile)
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_tensor=Input(shape=input_shape)
    )
    
    # Base model katmanlarını dondur
    for layer in base_model.layers:
        layer.trainable = False
    
    # Özel katmanlar ekle
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    # Final modeli oluştur
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Modeli compile et
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ Model başarıyla oluşturuldu!")
    return model

# Model parametreleri
IMG_SIZE = (224, 224, 3)
LEARNING_RATE = 0.001

# Modeli oluştur
model = create_model(input_shape=IMG_SIZE, learning_rate=LEARNING_RATE)

# Model özetini göster
print("\\n📊 Model Özeti:")
model.summary()

# Model mimarisini görselleştir
def plot_model_architecture():
    """
    Model mimarisini görselleştirir.
    """
    print("\\n🏗️ Model Mimarisi:")
    print("MobileNetV2 (include_top=False, weights='imagenet', frozen)")
    print("         ↓")
    print("GlobalAveragePooling2D")
    print("         ↓")
    print("Dense(128, activation='relu')")
    print("         ↓")
    print("Dropout(0.3)")
    print("         ↓")
    print("Dense(1, activation='sigmoid')")
    
    # Model parametrelerini göster
    total_params = model.count_params()
    print(f"\\n📊 Toplam parametre sayısı: {total_params:,}")

plot_model_architecture()

## 7. Modelin Eğitilmesi

Modelimizi eğitim verisiyle eğitelim ve eğitim sürecini takip edelim.

In [ ]:
# Eğitim parametreleri
EPOCHS = 15
BATCH_SIZE = 32

# Callbacks tanımla
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5,
        min_lr=0.000001,
        verbose=1
    )
]

print("🚀 Model eğitimi başlıyor...")
print(f"📊 Eğitim parametreleri:")
print(f"   • Epoch sayısı: {EPOCHS}")
print(f"   • Batch boyutu: {BATCH_SIZE}")
print(f"   • Öğrenme oranı: {LEARNING_RATE}")

# Eğitim verisini hazırla
if 'X_train' in locals():
    # Eğitim verilerini normalize et (veri artırma için)
    X_train_norm = X_train.astype('float32') / 255.0
    X_test_norm = X_test.astype('float32') / 255.0
    
    # Modeli eğit
    history = model.fit(
        train_datagen.flow(X_train_norm, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(X_train_norm) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_datagen.flow(X_test_norm, y_test, batch_size=BATCH_SIZE),
        validation_steps=len(X_test_norm) // BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    
    print("✅ Model eğitimi tamamlandı!")
    
    # Modeli kaydet
    model.save('face_mask_detector.h5')
    print("💾 Model kaydedildi: face_mask_detector.h5")
    
    # Eğitim geçmişini kaydet
    with open('training_history.pkl', 'wb') as f:
        pickle.dump(history.history, f)
    print("💾 Eğitim geçmişi kaydedildi: training_history.pkl")
    
else:
    print("❌ Eğitim verisi hazırlanmadı!")

## 8. Modelin Değerlendirilmesi

Eğitilmiş modelimizi test verisiyle değerlendirelim ve performans metriklerini inceleyelim.

In [ ]:
# Eğitim geçmişini görselleştir
def plot_training_history(history):
    """
    Eğitim geçmişini görselleştirir.
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Doğruluk grafiği
    axes[0].plot(history['accuracy'], label='Eğitim Doğruluğu', color='blue')
    axes[0].plot(history['val_accuracy'], label='Doğrulama Doğruluğu', color='red')
    axes[0].set_title('Model Doğruluğu', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Doğruluk')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Kayıp grafiği
    axes[1].plot(history['loss'], label='Eğitim Kaybı', color='blue')
    axes[1].plot(history['val_loss'], label='Doğrulama Kaybı', color='red')
    axes[1].set_title('Model Kaybı', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Kayıp')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Model performansını değerlendir
def evaluate_model(model, X_test, y_test):
    """
    Model performansını değerlendirir.
    """
    print("🔍 Model değerlendiriliyor...")
    
    # Test verisi üzerinde tahmin yap
    predictions = model.predict(X_test, verbose=0)
    y_pred = (predictions > 0.5).astype(int).flatten()
    
    # Metrikleri hesapla
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"✅ Model değerlendirme sonuçları:")
    print(f"   • Doğruluk (Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   • Kesinlik (Precision): {precision:.4f}")
    print(f"   • Duyarlılık (Recall): {recall:.4f}")
    print(f"   • F1 Skoru: {f1:.4f}")
    
    return y_pred, predictions

# Confusion Matrix'i görselleştir
def plot_confusion_matrix(y_true, y_pred, classes=['No Mask', 'Mask']):
    """
    Confusion matrix'i görselleştirir.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('Gerçek Etiket', fontsize=12)
    plt.xlabel('Tahmin Edilen Etiket', fontsize=12)
    plt.show()
    
    return cm

# Değerlendirmeleri çalıştır
if 'history' in locals() and 'X_test' in locals():
    # Eğitim geçmişini görselleştir
    print("📊 Eğitim geçmişi:")
    plot_training_history(history.history)
    
    # Model performansını değerlendir
    y_pred, predictions = evaluate_model(model, X_test_norm, y_test)
    
    # Confusion Matrix
    print("\\n🔍 Confusion Matrix:")
    cm = plot_confusion_matrix(y_test, y_pred)
    
    # Detaylı sınıflandırma raporu
    print("\\n📋 Detaylı Sınıflandırma Raporu:")
    print(classification_report(y_test, y_pred, target_names=['No Mask', 'Mask']))
    
else:
    print("❌ Model henüz eğitilmedi!")

## 9. Model ile Tahmin ve Sonuçların Görselleştirilmesi

Eğitilmiş modelimizi kullanarak test görselleri üzerinde tahmin yapalım ve sonuçları görselleştirelim.

In [ ]:
# Test görüntüleri üzerinde tahmin görsellestir
def plot_predictions(model, X_test, y_test, num_samples=10):
    """
    Test görüntüleri üzerinde tahminleri görselleştirir.
    """
    # Rastgele örnekler seç
    indices = np.random.choice(len(X_test), num_samples, replace=False)
    
    # Tahmin yap
    predictions = model.predict(X_test[indices], verbose=0)
    
    # Görselleştir
    plt.figure(figsize=(20, 8))
    
    for i, idx in enumerate(indices):
        plt.subplot(2, 5, i+1)
        
        # Görüntüyü normalize et
        img = X_test[idx].copy()
        img = (img - img.min()) / (img.max() - img.min())
        
        # Tahmin ve gerçek etiketleri al
        pred_prob = predictions[i][0]
        pred_label = "Mask" if pred_prob > 0.5 else "No Mask"
        true_label = "Mask" if y_test[idx] == 1 else "No Mask"
        
        # Doğruluk kontrolü
        correct = (pred_prob > 0.5) == (y_test[idx] == 1)
        color = 'green' if correct else 'red'
        
        # Görselleştir
        plt.imshow(img)
        plt.title(f'Gerçek: {true_label}\\nTahmin: {pred_label}\\nOlasılık: {pred_prob:.2f}', 
                 color=color, fontsize=10, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Tahmin fonksiyonu
def predict_single_image(model, image_path):
    """
    Tek bir görüntü üzerinde tahmin yapar.
    """
    try:
        # Görüntüyü yükle
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Görüntü yüklenemedi: {image_path}")
        
        # Ön işlemden geçir
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        
        # Tahmin yap
        prediction = model.predict(img, verbose=0)[0][0]
        
        # Sonucu döndür
        if prediction > 0.5:
            return "Mask", prediction
        else:
            return "No Mask", 1 - prediction
            
    except Exception as e:
        print(f"Hata: {e}")
        return None, None

# Görselleştirmeleri çalıştır
if 'model' in locals() and 'X_test' in locals():
    print("🎯 Test görüntüleri üzerinde tahmin örnekleri:")
    plot_predictions(model, X_test_norm, y_test, num_samples=10)
    
    # Model özeti
    print("\\n📊 Son Model Özeti:")
    print("="*50)
    print(f"🏗️ Mimari: MobileNetV2 + Transfer Learning")
    print(f"📏 Girdi boyutu: {IMG_SIZE}")
    print(f"🎯 Sınıf sayısı: 2 (Mask, No Mask)")
    print(f"⚙️ Optimizer: Adam")
    print(f"📉 Loss: Binary Crossentropy")
    print(f"🎓 Eğitim epoch: {EPOCHS}")
    print(f"📦 Batch boyutu: {BATCH_SIZE}")
    
    if 'history' in locals():
        final_accuracy = history.history['val_accuracy'][-1]
        final_loss = history.history['val_loss'][-1]
        print(f"✅ Final doğruluk: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
        print(f"📉 Final kayıp: {final_loss:.4f}")
    
    print("="*50)
    print("✅ Proje başarıyla tamamlandı!")
    print("💾 Model 'face_mask_detector.h5' olarak kaydedildi.")
    print("🚀 Artık modeli gerçek zamanlı tespit için kullanabilirsiniz!")
    
else:
    print("❌ Model henüz eğitilmedi!")

## 🎉 Sonuç ve İleriye Dönük Adımlar

Bu projede MobileNetV2 mimarisi kullanarak başarılı bir yüz maskesi tespit sistemi geliştirdik. 

### 📈 Başarılan Hedefler:
- ✅ Kaggle veri setinden yüz maskesi verilerini işledik
- ✅ MobileNetV2 ile transfer learning uyguladık
- ✅ Veri artırma teknikleri ile model performansını artırdık
- ✅ Comprehensive model evaluation ve görselleştirme yaptık
- ✅ Gerçek zamanlı tespit için model kaydettik

### 🚀 Sonraki Adımlar:
1. **Gerçek Zamanlı Tespit**: Webcam ile canlı tespit yapın
2. **Model İyileştirme**: Fine-tuning ve hyperparameter optimization
3. **Deployment**: Web uygulaması veya mobile app geliştirin
4. **Genişletme**: Farklı maske türlerini tespit edin

### 📝 Kullanım:
```python
# Gerçek zamanlı tespit için:
python src/detect.py

# Tek görüntü üzerinde tahmin için:
python src/evaluate.py path/to/image.jpg

# Model eğitimi için:
python src/train.py
```

### 🔗 Proje Dosyaları:
- `face_mask_detector.h5`: Eğitilmiş model
- `training_history.pkl`: Eğitim geçmişi
- `src/`: Kaynak kod dosyaları
- `data/`: Veri seti klasörü

**Teşekkürler!** 🎭